In [1]:
import sys
import os
from pathlib import Path
sys.path.append(os.path.join(Path.home(), 'QUIC-Projects'))
sys.path

['/home/quic/QUIC-Projects/summary_ipynb',
 '/home/quic/.vscode-server/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles/vscode_datascience_helpers/../.does-not-exist',
 '/home/quic/.vscode-server/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles',
 '/home/quic/.vscode-server/extensions/ms-toolsai.jupyter-2020.12.414227025/pythonFiles/lib/python',
 '/home/quic/anaconda3/envs/quic/lib/python38.zip',
 '/home/quic/anaconda3/envs/quic/lib/python3.8',
 '/home/quic/anaconda3/envs/quic/lib/python3.8/lib-dynload',
 '',
 '/home/quic/anaconda3/envs/quic/lib/python3.8/site-packages',
 '/home/quic/anaconda3/envs/quic/lib/python3.8/site-packages/IPython/extensions',
 '/home/quic/.ipython',
 '/home/quic/QUIC-Projects']

In [4]:
from custom_qiskit_v3_1.datageneration import Sklearn_DataLoader
from sklearn.datasets import load_iris
dl = Sklearn_DataLoader('iris', labels=(0,1))
X, y, Xt, yt = dl(2**6, 0)


In [5]:
dl.lb.classes_

array([0, 1])